In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.stem.snowball import SnowballStemmer
from gensim import corpora, models
import gensim

In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ddm1924/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [7]:
sc = SparkContext(appName="SDDM")
# get the number of executers and cores
# print(sc._jsc.sc().getExecutorMemoryStatus())
# submit Sparkjobs
# ./bin/spark-submit --driver-memory 4g --executor-memory 2G --master spark://fs.das3.liacs.nl:7077 sample_pyspark.py

In [9]:
sc = SparkContext.getOrCreate()

In [10]:
sc.master

'local[*]'

In [11]:
from pyspark.sql import SparkSession

In [12]:
ss = SparkSession.builder.getOrCreate()

In [13]:
import pandas as pd

In [14]:
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield (eval(l))
        
def GetDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient = "index")

df = GetDF("reviews_Sports_and_Outdoors_5.json.gz")

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5.0,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5.0,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4.0,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4.0,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4.0,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"


In [13]:
df = df[['reviewText', 'summary']]

In [14]:
sp_df = SQLContext(sc).createDataFrame(df)

In [15]:
sp_df = sp_df.na.drop()

In [16]:
sp_df.show(5)

+--------------------+--------------------+
|          reviewText|             summary|
+--------------------+--------------------+
|This came in on t...|      Woks very good|
|I had a factory G...|Works as well as ...|
|If you don't have...|It's a punch, tha...|
|This works no bet...|It's a punch with...|
|I purchased this ...|Ok,tool does what...|
+--------------------+--------------------+
only showing top 5 rows



In [17]:
sp_df.select('reviewText').first()

Row(reviewText='This came in on time and I am veru happy with it, I haved used it already and it makes taking out the pins in my glock 32 very easy')

In [18]:
rdd = sp_df.select('reviewText').rdd

In [19]:
tokenizer = RegexpTokenizer(r'\w+')
nltk_stpwd = stopwords.words("english")
stop_words_stpwd = get_stop_words('en')
merged_stopwords = list(set(nltk_stpwd + stop_words_stpwd))
sb_stemmer = SnowballStemmer('english')

In [20]:
def dropStopWords(s):
    tokens = tokenizer.tokenize(s[0])
    stopped_tokens = [token for token in tokens if token not in merged_stopwords]
    stemmed_tokens = [sb_stemmer.stem(token) for token in stopped_tokens]
    
    # add tokens to list
    return(stemmed_tokens)

In [21]:
texts_rdd = rdd.flatMap(dropStopWords)

In [8]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

In [9]:
# Load and parse the data
data = sc.textFile("/home/ddm1924/spark/data/mllib/sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

In [18]:
parsedData.first()

DenseVector([1.0, 2.0, 6.0, 0.0, 2.0, 3.0, 1.0, 1.0, 0.0, 0.0, 3.0])

In [24]:
# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())+ " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 8.605904408398544
 12.909497901537755
 4.972791291787081
 4.812989170255772
 16.231428936190717
 6.296126604096916
 5.4738409512965385
 4.407080087161346
 3.3859620662775245
 11.71424428022948
 6.58118415293947
Topic 1:
 4.969888599351531
 3.234325908093829
 1.423420528714697
 32.29408913417656
 3.398065965889449
 3.267226102097135
 9.421411557650572
 1.005248294078012
 2.239044621393636
 5.225087874983252
 22.043814686429393
Topic 2:
 12.424206992249927
 12.856176190368418
 5.603788179498222
 2.8929216955676678
 5.370505097919837
 12.43664729380595
 16.10474749105289
 4.587671618760642
 2.374993312328839
 7.06066784478727
 4.375001160631137


In [21]:
# Save and load model
ldaModel.save(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")
sameModel = LDAModel.load(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")

In [25]:
ldaModel.describeTopics()

[([4, 1, 9, 0, 10, 5, 6, 2, 3, 7, 8],
  [0.1900834919428993,
   0.15118092498205635,
   0.1371835139722902,
   0.10078227663787527,
   0.07707112354850948,
   0.07373286328185714,
   0.06410321644834033,
   0.05823550946513061,
   0.056364094114087375,
   0.05161056217125915,
   0.039652423435694986]),
 ([3, 10, 6, 9, 0, 4, 5, 1, 8, 2, 7],
  [0.36481582623754666,
   0.24902180813473992,
   0.10643062349421809,
   0.05902611906333325,
   0.056143215811038635,
   0.03838684651562803,
   0.03690879111001245,
   0.03653712831411121,
   0.02529375918121191,
   0.0160799189631573,
   0.011355963175002403]),
 ([6, 1, 5, 0, 9, 2, 4, 7, 10, 3, 8],
  [0.18707454482897956,
   0.14933877792188216,
   0.14446548342219317,
   0.1443209754904531,
   0.08201750595506141,
   0.06509422911349906,
   0.06238438679359784,
   0.05329090570226417,
   0.05082050191758821,
   0.03360450138847967,
   0.027588187466001727])]